In [1]:
# Importing requird python packages
# Path to hsmpy package
HSMPY_PATH = r'\\Chcfpp01\Groups\HTS\Code_Repository\Python\Libraries'
Site_Packages = r'c:\users\mr068144\appdata\roaming\python\python36\site-packages'
import sys, os, arcpy, csv, json, math
print(sys.executable)
print(sys.version)
for p in [HSMPY_PATH,Site_Packages]:
    if not p in sys.path:
        sys.path.append(p)
import hsmpy3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
from IPython.display import Image
from IPython.core.display import HTML 
import subprocess 
import webbrowser
import time
import copy
import zipfile
import shutil
import scipy
from ipywidgets import IntProgress
from time import gmtime, strftime
#from pysal.esda.mapclassify import Natural_Breaks as nb
import matplotlib.patches as patches
print(strftime("%Y-%m-%d %H:%M:%S"))

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\python.exe
3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
2019-04-01 16:24:50


In [2]:
# Defining Parameteres
print(strftime("%Y-%m-%d %H:%M:%S"))
Years = range(2012,2017)
MaxOpenProcesses = 15
Project_DIR  = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO00_OtherStates\NewMexico\3_WorkData\7_NetworkScreening'
ReceivedDir  = Project_DIR + '\\2.Received_Data'
Project_GDB  = Project_DIR + '\\5.NM_NS_GDB.gdb'
PythonDir   = Project_DIR + '\\6.Python_Notebook'

Src_Loc  = ReceivedDir + '\\crash_asof_11jun2018.csv'
Src_Unit = ReceivedDir + '\\vehicle_asof_11jun2018.csv'
Src_Occ  = ReceivedDir + '\\occupant_asof_11jun2018.csv'

Src_Ped_Loc  = ReceivedDir + '\\Ped_2017_KAB\\crashkabped.csv'
Src_Ped_Unit = ReceivedDir + '\\Ped_2017_KAB\\vehiclekabped.csv'
Src_Ped_Occ  = ReceivedDir + '\\Ped_2017_KAB\\occkabped.csv'

HPMS105 = {year:ReceivedDir + '\\v105\\jg_request.gdb\\HPMS{}Data'.format(year) for year in Years}
HPMS10  = {year:ReceivedDir + '\\v10\\jg_request.gdb\\HPMS{}Data'.format(year) for year in Years}

FOU_2017_105 = ReceivedDir + '\\v105\\jg_request.gdb\\HPMS2017Ownership_Urban_FuncSys'
FOU_2017_10  = ReceivedDir + '\\v10\\jg_request.gdb\\HPMS2017Ownership_Urban_FuncSys'

FHWA_HPMS     = {year:ReceivedDir + '\\FHWA_HPMS\\newmexico{}\\NewMexico{}.shp'.format(year,year) for year in Years}
MS2_HPMS_2017 = {year:ReceivedDir + '\\NM_HPMS_N2017_A{}.csv'.format(year) for year in Years}

Routes  = {year:Project_GDB + '\\Routes_{}'.format(year) for year in Years}
AttTabs = {year:Project_GDB + '\\SegAtt_{}'.format(year) for year in Years}
Segs    = {year:Project_GDB + '\\Seg_{}'.format(year) for year in Years}
CrashData = {year:Project_GDB + '\\Crash_{}'.format(year) for year in range(2012,2018)}

StreetNames_SHP = r"\\chcfpp01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO00_OtherStates\NewMexico\3_WorkData\7_NetworkScreening\2.Received_Data\streetNames\AltStreetName.shp"

UTM13N   = arcpy.SpatialReference(26913)
NAD83_NM = arcpy.SpatialReference(6529)

RouteID = "ROUTE_ID"
BEG_STA = 'BEG_POINT'
END_STA = 'END_POINT'
Fields = ['F_SYSTEM','THROUGH_LANES','FACILITY_TYPE','AADT','MEDIAN_TYPE','SPEED_LIMIT','LANE_WIDTH']

2019-04-01 16:24:50


In [95]:
HPMS_DF = hsmpy3.common.FCtoDF(Segs[2016],readGeometry=True)

In [99]:
# Reading Source Crashes
Loc_DF = pd.read_csv(Src_Loc,low_memory=False)
Loc_DF['CID'] = ['{}_{}'.format(y,u) for y,u in zip(Loc_DF.year,Loc_DF.UCRnumber)]
#Loc_DF.to_excel(ReceivedDir + '\\Loc_Input.xlsx',index = False)
#arcpy.Delete_management(Project_GDB + '\\Loc_Input')
#arcpy.TableToTable_conversion(in_rows=ReceivedDir + '\\Loc_Input.xlsx',out_path=Project_GDB,out_name='Loc_Input')
Loc_DF.index = ['{}_{}'.format(y,u) for y,u in zip(Loc_DF.year,Loc_DF.UCRnumber)]
Loc_DF = Loc_DF.sort_index()

Unit_DF = pd.read_csv(Src_Unit,low_memory=False)
Unit_DF['CUID'] = ['{}_{}_{}'.format(y,u,v) for y,u,v in zip(Unit_DF.year, Unit_DF.UCRnumber,Unit_DF.VehNo)]
Unit_DF['CID'] = ['{}_{}'.format(y,u) for y,u in zip(Unit_DF.year, Unit_DF.UCRnumber)]
#Unit_DF.to_excel(ReceivedDir + '\\Unit_Input.xlsx',index = False)
#arcpy.Delete_management(Project_GDB + '\\Unit_Input')
#arcpy.TableToTable_conversion(in_rows=ReceivedDir + '\\Unit_Input.xlsx',out_path=Project_GDB,out_name='Unit_Input')
Unit_DF.index = pd.MultiIndex.from_tuples([('{}_{}'.format(y,u),v) for y,u,v in zip(Unit_DF.year, Unit_DF.UCRnumber,Unit_DF.VehNo)],names=['CID','Unit'])
Unit_DF = Unit_DF.sort_index()

OL = ['CUID','PPLNo', 'OccNo', 'SeatPos', 'Age', 'Sex', 'Injury', 'OPCode', 'OPProperlyUsed', 'AirbagDeployed', 'Ejected', 'MedTrans', 'DAparked', 'Belt', 'Helmet']
Occ_DF = pd.read_csv(Src_Occ,low_memory=False)
Occ_DF['CUID'] = ['{}_{}_{}'.format(y,u,v) for y,u,v,p in zip(Occ_DF.year, Occ_DF.UCRnumber,Occ_DF.VehNo,Occ_DF.PPLNo)]
#Occ_DF[OL].to_csv(ReceivedDir + '\\Occ_Input.csv',index = False)
#arcpy.Delete_management(Project_GDB + '\\Occ_Input')
#arcpy.TableToTable_conversion(in_rows=ReceivedDir + '\\Occ_Input.csv',out_path=Project_GDB,out_name='Occ_Input')
Occ_DF.index = pd.MultiIndex.from_tuples([('{}_{}'.format(y,u),v,p) for y,u,v,p in zip(Occ_DF.year, Occ_DF.UCRnumber,Occ_DF.VehNo,Occ_DF.PPLNo)],names=['CID','Unit','Person'])
Occ_DF = Occ_DF.sort_index()

In [100]:
# Reading NM Crash Data
Crash_DF = []
for y in range(2012,2018):
    df = hsmpy3.common.FCtoDF(CrashData[y])
    df['YEAR'] = y
    Crash_DF.append(df)
Crash_DF = pd.concat(Crash_DF)
Crash_DF.loc[Crash_DF.YEAR==2017,'CID'] = Crash_DF.loc[Crash_DF.YEAR==2017,'UCRnum'].astype(int).apply(lambda x:'{}_{}'.format(2017,x))
Crash_DF.index = Crash_DF.CID

In [185]:
arcpy.AddField_management(Ped_FC,'SiteID','LONG')
uc = arcpy.UpdateCursor(Ped_FC)
for row in uc:
    r = row.getValue('FID')
    row.setValue('SiteID',r+1)
    uc.updateRow(row)
del uc
del row

In [3]:
# Reading the Final Network Screening Results
SHP_Dir = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO00_OtherStates\NewMexico\3_WorkData\7_NetworkScreening\8.Deliverable\Shapefiles'
Ped_FC = SHP_Dir + '\\State_Pedestrian.shp' 
Seg_FC = SHP_Dir + '\\State_Segment.shp' 
Int_FC = SHP_Dir + '\\State_Int.shp'
IntD_DF = hsmpy3.common.FCtoDF(Int_FC,readGeometry=True)
SegD_DF = hsmpy3.common.FCtoDF(Seg_FC,readGeometry=True)
PedD_DF = hsmpy3.common.FCtoDF(Ped_FC,readGeometry=True)
SegD_DF['Mileage'] = SegD_DF.Shape.apply(lambda x:x.length/5280.0)
PedD_DF['Mileage'] = PedD_DF.Shape.apply(lambda x:x.length/5280.0)
IntD_DF = IntD_DF.sort_values('CF_Obs',ascending=False)
SegD_DF = SegD_DF.reset_index()
idx = pd.IntervalIndex.from_tuples([(bmp,emp) for bmp,emp in zip(SegD_DF.BMP,SegD_DF.EMP)],'left')
SegD_DF.index = pd.MultiIndex.from_arrays([SegD_DF.RID.apply(lambda x:x.replace('-','')),idx],names = ['Route ID','Milepost'])
SegD_DF = SegD_DF.sort_index()
IntD_DF = IntD_DF[IntD_DF.District.apply(len)>1]

In [5]:
df1 = pd.DataFrame(SegD_DF.groupby(['District','SafetyTier']).CF_Obs.sum()).unstack().fillna(0).astype(int).iloc[1:]
df2 = pd.DataFrame(IntD_DF.groupby(['District','SafetyTier']).CF_Obs.sum()).unstack().fillna(0).astype(int)
df3 = pd.DataFrame(PedD_DF.groupby(['District','SafetyTier']).CF_Obs.sum()).unstack().fillna(0).astype(int)    
for df,n in zip([df1,df2,df3],['Segments','Intersections','Pedestrians']):
    df.columns = pd.MultiIndex.from_product([[n],['Tier {}'.format(i) for i in '123']])

df = pd.concat([df1,df2,df3],axis=1).iloc[1:].fillna(0).astype(int)    
df = df.swaplevel(axis=1,i=1,j=0)
df.sortlevel(axis=1)

Tier 1                             Tier 2              \
           Intersections Pedestrians Segments Intersections Pedestrians   
District 1           429           0       77           519          76   
District 2           186           0      167           565          29   
District 3          2813         282      318          1995         613   
District 4             0           0        0            20           2   
District 5           337           9      205           682         107   
District 6            43          10       68            97          90   

                           Tier 3                       
           Segments Intersections Pedestrians Segments  
District 1      275          1082         173      290  
District 2      558          1705         163      615  
District 3      332          2403         498      225  
District 4       55           211          28      231  
District 5      475          1247         206      480  
District 6      206           295          90      250

In [46]:
# Read Street Names
def DissolveDF(DF,RID,BMP,EMP,DissFields):
    print('[{}] dissolving by milepost: {}'.format(strftime("%Y-%m-%d %H:%M:%S"),DissFields))
    print('[{}] sorting and indexing:'.format(strftime("%Y-%m-%d %H:%M:%S")))
    F1 = [RID,BMP,EMP]
    F2 = DissFields
    df = DF[F1+F2]
    df.index = range(0,df.shape[0])
    df = df.sort_values(by=[RID])
    df = df.fillna(-1)
    print('[{}]  - {}:'.format(strftime("%Y-%m-%d %H:%M:%S"),df.shape))
    print('[{}] grouping intervals:'.format(strftime("%Y-%m-%d %H:%M:%S")))
    idx = df.groupby([RID] + F2)[BMP].shift(-1) != df[EMP]
    df['EMP2'] = df.loc[idx, EMP]
    df['EMP2'] = df.groupby([RID] + F2)['EMP2'].fillna(method='backfill')
    df['EMP2'] = df['EMP2'].fillna(df[EMP]) 
    print('[{}] aggregating groups:'.format(strftime("%Y-%m-%d %H:%M:%S")))
    sdf = df.groupby([RID] + F2 + ['EMP2'], as_index=False).agg({BMP: 'first', EMP: 'last'}).drop(['EMP2'], axis=1)
    print('[{}]  - {}:'.format(strftime("%Y-%m-%d %H:%M:%S"),sdf.shape))

    sdf = sdf[F1+F2].sort_values(F1)
    print('[{}] done!'.format(strftime("%Y-%m-%d %H:%M:%S")))
    return(sdf)

print('[{}] read street names'.format(strftime("%Y-%m-%d %H:%M:%S")))
SN_DF = hsmpy3.common.FCtoDF(Project_GDB+'\\StreetNames_Projected',selectedFields=['RouteID','FromMeasur','ToMeasure','RoadLabel'],readGeometry=True)
SN_DF.rename(columns={'RouteID':'RID','FromMeasur':'BMP','ToMeasure':'EMP'},inplace=True)
SN_DF.BMP = SN_DF.BMP.round(3)
SN_DF.EMP = SN_DF.EMP.round(3)
#SN_DF = DissolveDF(SN_DF,'RID','BMP','EMP',['RoadLabel'])
SN_DF = SN_DF[~pd.isnull(SN_DF.RoadLabel)]
#idx = pd.IntervalIndex.from_tuples([(bmp,emp) for bmp,emp in zip(SN_DF.BMP,SN_DF.EMP)],'left')
#SN_DF.index = pd.MultiIndex.from_arrays([SN_DF.RID,idx],names = ['Route ID','Milepost'])
SN_DF = SN_DF[SN_DF.RoadLabel.apply(lambda x:len(x.replace(' ',''))>1)]
SN_DF = SN_DF[SN_DF.RID.apply(lambda x:len(x.replace(' ',''))>1)]
SN_DF = SN_DF.sort_values(['RID','BMP','EMP'])
SN_DF.index = SN_DF.RID
SN_DF.shape

[2019-04-02 09:09:49] read street names


(84731, 5)

In [164]:
# Assign Segments StreetNames
try:
    arcpy.Delete_management(Project_GDB+'\\StreetNames')
except:
    pass
arcpy.CopyRows_management(in_rows=StreetNames_SHP,out_table=Project_GDB+'\\StreetNames')
try:
    arcpy.Delete_management(Project_GDB+'\\SegmentsPSI')
except:
    pass
arcpy.CopyRows_management(in_rows=Seg_FC,out_table=Project_GDB+'\\SegmentsPSI')
uc = arcpy.UpdateCursor(Project_GDB+'\\SegmentsPSI')
for row in uc:
    r = row.getValue('RID')
    row.setValue('RID',r.replace('-',''))
    uc.updateRow(row)
del uc
del row
try:
    arcpy.Delete_management(Project_GDB+'\\Seg_StNames')
except:
    pass
arcpy.lr.OverlayRouteEvents(
    in_table=Project_GDB+'\\SegmentsPSI',
    overlay_table=Project_GDB+'\\StreetNames',
    out_table=Project_GDB+'\\Seg_StNames',
    in_event_properties='RID LINE BMP EMP',
    overlay_event_properties='RouteID LINE FromMeasur ToMeasure',
    out_event_properties='RID LINE BMP EMP',
    overlay_type='UNION',
)
uc = arcpy.UpdateCursor(Project_GDB+'\\Seg_StNames')
for row in uc:
    r = row.getValue('SiteID')
    if not r>0:
        uc.deleteRow(row)
del uc
del row

SegD_DF.index = SegD_DF.SiteID
df = hsmpy3.common.FCtoDF(Project_GDB+'\\Seg_StNames',selectedFields=['SiteID','RoadLabel'])
SegD_DF['RoadLabel'] = df.groupby('SiteID').RoadLabel.apply(lambda x:';'.join([i for i in list(set(x.tolist())) if len(i)>0])).loc[SegD_DF.index]

arcpy.AddField_management(Seg_FC,'RoadLabel','TEXT',100)
uc = arcpy.UpdateCursor(Seg_FC)
for row in uc:
    r = row.getValue('SiteID')
    row.setValue('RoadLabel',SegD_DF.RoadLabel.loc[r])
    uc.updateRow(row)
del uc
del row


<Result '\\\\CHCFPP01\\Proj\\ILDOT\\650511SAFETYPROGRAM\\4_WorkData\\WO00_OtherStates\\NewMexico\\3_WorkData\\7_NetworkScreening\\5.NM_NS_GDB.gdb\\Seg_StNames'>

In [187]:
# Assign Pedestrian Street Names
try:
    arcpy.Delete_management(Project_GDB+'\\StreetNames')
except:
    pass
arcpy.CopyRows_management(in_rows=StreetNames_SHP,out_table=Project_GDB+'\\StreetNames')

try:
    arcpy.Delete_management(Project_GDB+'\\SegmentsPSI')
except:
    pass
arcpy.CopyRows_management(in_rows=Ped_FC,out_table=Project_GDB+'\\SegmentsPSI')
uc = arcpy.UpdateCursor(Project_GDB+'\\SegmentsPSI')
for row in uc:
    r = row.getValue('RID')
    row.setValue('RID',r.replace('-',''))
    uc.updateRow(row)
del uc
del row
try:
    arcpy.Delete_management(Project_GDB+'\\Seg_StNames')
except:
    pass
arcpy.lr.OverlayRouteEvents(
    in_table=Project_GDB+'\\SegmentsPSI',
    overlay_table=Project_GDB+'\\StreetNames',
    out_table=Project_GDB+'\\Seg_StNames',
    in_event_properties='RID LINE BMP EMP',
    overlay_event_properties='RouteID LINE FromMeasur ToMeasure',
    out_event_properties='RID LINE BMP EMP',
    overlay_type='UNION',
)
uc = arcpy.UpdateCursor(Project_GDB+'\\Seg_StNames')
for row in uc:
    r = row.getValue('SiteID')
    if not r>0:
        uc.deleteRow(row)
del uc
del row

PedD_DF.index = PedD_DF.SiteID
df = hsmpy3.common.FCtoDF(Project_GDB+'\\Seg_StNames',selectedFields=['SiteID','RoadLabel'])
PedD_DF['RoadLabel'] = df.groupby('SiteID').RoadLabel.apply(lambda x:';'.join([i for i in list(set(x.tolist())) if len(i)>0])).loc[PedD_DF.index]

arcpy.AddField_management(Ped_FC,'RoadLabel','TEXT',100)
uc = arcpy.UpdateCursor(Ped_FC)
for row in uc:
    r = row.getValue('SiteID')
    row.setValue('RoadLabel',PedD_DF.RoadLabel.loc[r])
    uc.updateRow(row)
del uc
del row

In [75]:
# Assign intersection Street Names
IntD_DF.index = IntD_DF.Int_ID
def Extract(row):
    ridL = row.RIDList
    k = 0
    p1 = row.Shape
    ridL = ridL.split(';')
    ridL = list(set([i.replace('-','') for i in ridL if i[-1]=='P']))
    #print(row.Int_ID,ridL)
    res = [-1,-1,-1]
    df = pd.DataFrame(columns=['Int_ID','RIDList','RID','BMP','EMP','MP','Offset','StreetName'])
    for rid in ridL:
        if rid in SN_DF.RID:
            offset = -1
            for i,r in SN_DF.loc[[rid]].iterrows():
                pl = r.Shape
                if pl.length>0:
                    res = pl.queryPointAndDistance(p1)     
                    #print('BMP:{:>10.3f},EMP:{:>10.3f},MP:{:>10.3f},Distance:{:>10.3f}'.
                    #      format(r.BMP,r.EMP,res[0].firstPoint.M,res[2]))
                    k += 1
                    df.loc[k] = [row.Int_ID,';'.join(ridL),rid,r.BMP,r.EMP,res[0].firstPoint.M,res[2],r.RoadLabel]
    df = df.sort_values(['RID','Offset'])
    df = df.groupby('RID').apply(lambda df:df[df.Offset==df.Offset.min()])
    #display(df)
    if df.shape[0]>0:
        return(';'.join(['{}@{:.3f}({})'.format(r.RID,r.MP,r.StreetName) for i,r in df.iterrows()]))
    else:
        return('')
IntD_DF['StreetName'] = IntD_DF.apply(Extract,axis=1)
arcpy.AddField_management(Int_FC,'ApprLabels','TEXT',300)
uc = arcpy.UpdateCursor(Int_FC)
for row in uc:
    r = row.getValue('Int_ID')
    if r in IntD_DF.index:
        row.setValue('ApprLabels',IntD_DF.StreetName.loc[r])
    else:
        print(r,'Not found')
    uc.updateRow(row)
del uc
del row

In [155]:
HPMS_DF.index = HPMS_DF.ROUTE_ID
HPMS_DF.BEG_POINT = HPMS_DF.BEG_POINT.round(4)
HPMS_DF.END_POINT = HPMS_DF.END_POINT.round(4)
def FindSeg(row):
    df = HPMS_DF.loc[[row.RID]]
    BMP = min([df.BEG_POINT[df.BEG_POINT-row.BMP<= 0.001].max(),row.BMP])
    EMP = max([df.END_POINT[df.END_POINT-row.EMP>=-0.001].min(),row.EMP])
    BMP = row.BMP if pd.isnull(BMP) else BMP
    EMP = row.EMP if pd.isnull(EMP) else EMP
    return([BMP,EMP])
SegD_DF['SegMPs'] = SegD_DF.apply(FindSeg,axis=1)
SegD_DF.SegMPs

SegID
BIA-AI0005-P_1.2000_1.4000                 [0.0, 12.5]
BIA-AI0005-P_1.4000_3.0000                 [0.0, 12.5]
BIA-AI0005-P_3.0000_3.2000                 [0.0, 12.5]
BIA-AI0005-P_4.4000_6.2000                 [0.0, 12.5]
BIA-AI0005-P_10.8000_12.5000               [0.0, 12.5]
BIA-AI009A-P_1.6000_3.4000               [0.0, 38.755]
BIA-AI009A-P_29.8000_31.6000             [0.0, 38.755]
BIA-MES004-P_0.0000_1.8000                 [0.0, 2.74]
BIA-NAV036-P_5.4000_6.2000                [0.0, 27.59]
BIA-NAV036-P_6.2000_7.2000                [0.0, 27.59]
BIA-NAV036-P_7.2000_8.0000                [0.0, 27.59]
BIA-NAV036-P_11.8000_14.6000              [0.0, 27.59]
BIA-NAV036-P_15.2000_17.0000              [0.0, 27.59]
BIA-NAV036-P_17.0000_17.4000              [0.0, 27.59]
BIA-NAV036-P_17.4000_18.4000              [0.0, 27.59]
BIA-NAV036-P_18.4000_19.2000              [0.0, 27.59]
BIA-NAV036-P_19.2000_20.0000              [0.0, 27.59]
BIA-NAV036-P_20.0000_20.6000              [0.0, 27.59]
BIA-

In [158]:
Crash_DF.index = Crash_DF.RID_P
def GetCrashCIDs_dep(row):
    df = Crash_DF.loc[[row.RID]]
    #display(df)
    adj = {True:0.899,False:0.199}[row.URBAN_CODE in [99999,99998,0]]
    df = df[(df.MP_P>=max([row.BMP-adj,row.SegMPs[0]])) & (df.MP_P<=min([row.EMP+adj,row.SegMPs[1]])) & (df.Int_Related==0)]
    return(df.CID.tolist())
def GetCrashCIDs(row):
    df = Crash_DF.loc[[row.RID]]
    #display(df)
    df = df[(df.MP_P>=row.BMP) & (df.MP_P<=row.EMP) & (df.Int_Related==0)]
    return(df.CID.tolist())
SegD_DF['CIDs'] = SegD_DF.apply(GetCrashCIDs,axis=1)
df = pd.DataFrame([(r.SiteID,cid) for i,r in SegD_DF.iterrows() for cid in r.CIDs],columns=['SiteID','CID'])
df.index = df.SiteID
df = df.sort_index()


State_Segment.shp,FID,RID,BMP,EMP,Length,FType,AADT_2016,F_SYSTEM,FACILITY_T,OWNERSHIP,...,SafetyTier,Shape_Leng,SiteID,RoadLabel,Shape,Mileage,SegID,CIDs,Diff,SegMPs
SegID,,,,,,,,,,,,,,,,,,,,,
NM-18-P_21.1330_25.2300,1907,NM-18-P,21.1330,25.2300,4.0970,R4U,3394,4,2,1,...,3,21632.760641,1908,NM HIGHWAY 18,(<geoprocessing array object object at 0x00000...,4.097114,NM-18-P_21.1330_25.2300,"[2012_30038651, 2014_30110019, 2014_30123187, ...",-3,"[8.933, 25.23]"
US-70-P_230.2710_233.0710,432,US-70-P,230.2710,233.0710,2.8000,R4U,5756,3,2,1,...,2,14778.847905,433,CENTRAL AVE;US HIGHWAY 70,(<geoprocessing array object object at 0x00000...,2.799024,US-70-P_230.2710_233.0710,"[2012_23225283, 2012_23300971, 2012_23301111, ...",-3,"[228.271, 251.486]"
NM-117-P_33.2000_36.0000,1543,NM-117-P,33.2000,36.0000,2.8000,R2U,304,5,2,1,...,3,14782.019575,1544,NM HIGHWAY 117,(<geoprocessing array object object at 0x00000...,2.799625,NM-117-P_33.2000_36.0000,"[2013_23181072, 2014_30064214, 2014_30105326, ...",-3,"[0.0, 56.726]"
NM-118-P_23.7400_24.2400,2748,NM-118-P,23.7400,24.2400,0.5000,U4D,14313,3,2,1,...,2,2626.931914,2749,INTERSTATE 40 BUSINESS,(<geoprocessing array object object at 0x00000...,0.497525,NM-118-P_23.7400_24.2400,"[2014_23166814, 2015_23411552, 2015_30129306, ...",-2,"[23.74, 25.742]"
US-380-P_220.0170_223.8170,2575,US-380-P,220.0170,223.8170,3.8000,R2U,1650,4,2,1,...,3,20011.823994,2576,W US HIGHWAY 380,(<geoprocessing array object object at 0x00000...,3.790118,US-380-P_220.0170_223.8170,"[2013_30092037, 2014_30092129, 2016_710377517]",-2,"[212.417, 226.032]"
US-70-P_363.6400_366.4400,1244,US-70-P,363.6400,366.4400,2.8000,R4D,1983,3,2,1,...,2,14779.016192,1245,CLOVIS HWY,(<geoprocessing array object object at 0x00000...,2.799056,US-70-P_363.6400_366.4400,"[2013_710163381, 2013_710165216, 2014_30106229...",-2,"[362.64, 383.523]"
US-70-P_358.1920_361.3920,104,US-70-P,358.1920,361.3920,3.2000,R4U,2129,3,2,1,...,3,16890.318956,105,CLOVIS HWY,(<geoprocessing array object object at 0x00000...,3.198924,US-70-P_358.1920_361.3920,"[2013_30063797, 2013_710166189, 2016_30167122]",-2,"[337.392, 362.64]"
US-64-P_271.7010_274.5010,1881,US-64-P,271.7010,274.5010,2.8000,R2U,2029,4,2,1,...,2,14766.208383,1882,US HIGHWAY 64,(<geoprocessing array object object at 0x00000...,2.796630,US-64-P_271.7010_274.5010,"[2012_30046065, 2014_23353048, 2015_30112433, ...",-2,"[255.901, 275.536]"
US-82-P_89.7840_93.5840,524,US-82-P,89.7840,93.5840,3.8000,R2U,608,4,2,1,...,3,19930.080155,525,HOPE HWY,(<geoprocessing array object object at 0x00000...,3.774636,US-82-P_89.7840_93.5840,"[2012_30023109, 2012_30050823, 2013_710165941]",-2,"[87.184, 104.01]"


In [184]:
#SegD_DF.apply(lambda s:pd.Series(s.CIDs,index=[s.index]*len(s.CIDs)),axis= 1)
df
#SegD_DF.groupby('SiteID').apply(lambda s:pd.DataFrame(s.CIDs,index=s.SegID))


,SiteID,CID
SiteID,,
1,1,2016_30179426
3,3,2014_30064722
4,4,2012_30038629
4,4,2013_30038706
4,4,2015_30124661
5,5,2015_23286823
6,6,2016_30181177
7,7,2014_30103081
8,8,2014_30109152


In [185]:
SegD_DF

State_Segment.shp,FID,RID,BMP,EMP,Length,FType,AADT_2016,F_SYSTEM,FACILITY_T,OWNERSHIP,...,SafetyTier,Shape_Leng,SiteID,RoadLabel,Shape,Mileage,SegID,CIDs,Diff,SegMPs
SegID,,,,,,,,,,,,,,,,,,,,,
BIA-AI0005-P_1.2000_1.4000,697,BIA-AI0005-P,1.200,1.400,0.200,R2U,287,6,2,0,...,3,1013.486689,698,,(<geoprocessing array object object at 0x00000...,0.191948,BIA-AI0005-P_1.2000_1.4000,[],1,"[0.0, 12.5]"
BIA-AI0005-P_1.4000_3.0000,785,BIA-AI0005-P,1.400,3.000,1.600,R2U,287,6,2,0,...,3,8107.816541,786,,(<geoprocessing array object object at 0x00000...,1.535571,BIA-AI0005-P_1.4000_3.0000,"[2016_23309695, 2016_23309699]",0,"[0.0, 12.5]"
BIA-AI0005-P_3.0000_3.2000,1850,BIA-AI0005-P,3.000,3.200,0.200,R2U,287,6,2,0,...,3,1013.586491,1851,,(<geoprocessing array object object at 0x00000...,0.191967,BIA-AI0005-P_3.0000_3.2000,[],1,"[0.0, 12.5]"
BIA-AI0005-P_4.4000_6.2000,1201,BIA-AI0005-P,4.400,6.200,1.800,R2U,287,6,2,0,...,3,9121.294697,1202,,(<geoprocessing array object object at 0x00000...,1.727518,BIA-AI0005-P_4.4000_6.2000,[2015_30153637],0,"[0.0, 12.5]"
BIA-AI0005-P_10.8000_12.5000,2436,BIA-AI0005-P,10.800,12.500,1.700,R2U,287,6,2,0,...,3,8614.611746,2437,,(<geoprocessing array object object at 0x00000...,1.631555,BIA-AI0005-P_10.8000_12.5000,[2015_23309628],0,"[0.0, 12.5]"
BIA-AI009A-P_1.6000_3.4000,948,BIA-AI009A-P,1.600,3.400,1.800,R2U,1398,5,2,0,...,3,9503.903234,949,,(<geoprocessing array object object at 0x00000...,1.799982,BIA-AI009A-P_1.6000_3.4000,[2016_30143413],0,"[0.0, 38.755]"
BIA-AI009A-P_29.8000_31.6000,508,BIA-AI009A-P,29.800,31.600,1.800,R2U,1398,5,2,0,...,3,9504.102534,509,,(<geoprocessing array object object at 0x00000...,1.800019,BIA-AI009A-P_29.8000_31.6000,[2014_23333928],0,"[0.0, 38.755]"
BIA-MES004-P_0.0000_1.8000,2357,BIA-MES004-P,0.000,1.800,1.800,R2U,314,6,2,0,...,3,9512.855731,2358,,(<geoprocessing array object object at 0x00000...,1.801677,BIA-MES004-P_0.0000_1.8000,[2012_23353101],0,"[0.0, 2.74]"
BIA-NAV036-P_5.4000_6.2000,368,BIA-NAV036-P,5.400,6.200,0.800,R2U,4803,4,2,0,...,3,4408.783145,369,,(<geoprocessing array object object at 0x00000...,0.834997,BIA-NAV036-P_5.4000_6.2000,[],1,"[0.0, 27.59]"
